In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
from openai import OpenAI
GPT_MODEL = "gpt-4o-mini"
client = OpenAI(api_key=OPENAI_API_KEY)

assistant = client.beta.assistants.create(
  name="Deal Stats Expert",
  instructions="You are a deals and crm expert. When provided with information, please distribute the information to the correct tool. If clarifications are needed as far as inputs, please do so.",
  model=GPT_MODEL,
  tools = [
    {
        "type": "function",
        "function": {
            "name": "get_win_rate",
            "description": "Get the win rate over a time period. We define win rate as won deals divided by total deals.",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The company that the user works for.",
                    },
                    "start_dt": {
                        "type": "string",
                        "description": "The start date of the date range for calculating win rate.",
                    },
                    "end_dt": {
                        "type": "string",
                        "description": "The end date of the date range for calculating win rate.",
                    },
                },
                "required": ["company_name", "start_dt", "end_dt"],
            },
        }
    },
]
)

In [3]:
thread = client.beta.threads.create()

In [4]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="What is my win rate?"
)

In [5]:
from typing_extensions import override
from openai import AssistantEventHandler
 
class EventHandler(AssistantEventHandler):
    @override
    def on_event(self, event):
      # Retrieve events that are denoted with 'requires_action'
      # since these will have our tool_calls
      if event.event == 'thread.run.requires_action':
        run_id = event.data.id  # Retrieve the run ID from the event data
        self.handle_requires_action(event.data, run_id)
 
    def handle_requires_action(self, data, run_id):
      tool_outputs = []
        
      for tool in data.required_action.submit_tool_outputs.tool_calls:
        if tool.function.name == "get_win_rate":
          tool_outputs.append({"tool_call_id": tool.id, "output": ".65"})
      # Submit all tool_outputs at the same time
      self.submit_tool_outputs(tool_outputs, run_id)
 
    def submit_tool_outputs(self, tool_outputs, run_id):
      # Use the submit_tool_outputs_stream helper
      with client.beta.threads.runs.submit_tool_outputs_stream(
        thread_id=self.current_run.thread_id,
        run_id=self.current_run.id,
        tool_outputs=tool_outputs,
        event_handler=EventHandler(),
      ) as stream:
        stream.until_done()
        # for text in stream.text_deltas:
        #   print(text, end="", flush=True)
        # print()
 
    @override
    def on_text_created(self, text) -> None:
      print(f"\nassistant > ", end="", flush=True)
        
    @override
    def on_text_delta(self, delta, snapshot):
      """
      Refers to any token creation essentially.
      """
      print(delta.value, end="", flush=True)
        
    def on_tool_call_created(self, tool_call):
      print(f"\nassistant > {tool_call.type}\n", flush=True)
    
    def on_tool_call_delta(self, delta, snapshot):
      if delta.type == 'code_interpreter':
        if delta.code_interpreter.input:
          print(delta.code_interpreter.input, end="", flush=True)
        if delta.code_interpreter.outputs:
          print(f"\n\noutput >", flush=True)
          for output in delta.code_interpreter.outputs:
            if output.type == "logs":
              print(f"\n{output.logs}", flush=True)
              
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  event_handler=EventHandler()
) as stream:
  stream.until_done()


assistant > Could you please provide the following details so I can calculate your win rate?

1. Your company's name.
2. The start date for the period you want to analyze.
3. The end date for that period.

In [6]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="I work at Twilio, and I want from the start of this quarter to the end of this quarter."
)

In [7]:
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  event_handler=EventHandler()
) as stream:
  stream.until_done()


assistant > To determine the start and end dates for this quarter (Q4 2023), please confirm the following dates:

- Start Date: October 1, 2023
- End Date: December 31, 2023

Shall I proceed with these dates?

In [8]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="Yes."
)

In [9]:
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  event_handler=EventHandler()
) as stream:
  stream.until_done()


assistant > function


assistant > Your win rate for Twilio from October 1, 2023, to December 31, 2023, is 65%.

In [11]:
for message in client.beta.threads.messages.list(thread.id):
    print(message.content)

[TextContentBlock(text=Text(annotations=[], value='Your win rate for Twilio from October 1, 2023, to December 31, 2023, is 65%.'), type='text')]
[TextContentBlock(text=Text(annotations=[], value='Yes.'), type='text')]
[TextContentBlock(text=Text(annotations=[], value='To determine the start and end dates for this quarter (Q4 2023), please confirm the following dates:\n\n- Start Date: October 1, 2023\n- End Date: December 31, 2023\n\nShall I proceed with these dates?'), type='text')]
[TextContentBlock(text=Text(annotations=[], value='I work at Twilio, and I want from the start of this quarter to the end of this quarter.'), type='text')]
[TextContentBlock(text=Text(annotations=[], value="Could you please provide the following details so I can calculate your win rate?\n\n1. Your company's name.\n2. The start date for the period you want to analyze.\n3. The end date for that period."), type='text')]
[TextContentBlock(text=Text(annotations=[], value='What is my win rate?'), type='text')]
